# Coursera: Data Analysis Tools – Assignment 3
Intro

Variables to explore:
* Cannabis use (transformed to quantitative, see Assignment 1)
* Tobacco use (to be transformed)

Question: is there any relationship between the amount of tobacco and cannabis use?

In [1]:
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
from scipy import stats

NESARC = r"C:\Users\USER\Documents\Courses\Coursera\dai\datasets\nesarc\nesarc_pds.csv"

# Load data
data = pd.read_csv(NESARC, low_memory=False)

Some variables regarding tobacco use
```
CHECK321 CIGARETTE SMOKING STATUS
9913 1. Smoked cigarettes in the past 12 months
8078 2. Smoked cigarettes prior to the last 12 months
22 9. Unknown
25080 BL. NA, never or unknown if ever smoked 100+ cigarettes

S3AQ3B1 USUAL FREQUENCY WHEN SMOKED CIGARETTES
14836 1. Every day
460 2. 5 to 6 Day(s) a week
687 3. 3 to 4 Day(s) a week
747 4. 1 to 2 Day(s) a week
409 5. 2 to 3 Day(s) a month
772 6. Once a month or less
102 9. Unknown
25080 BL. NA, never or unknown if ever smoked 100+ cigarettes


S3AQ3C1 USUAL QUANTITY WHEN SMOKED CIGARETTES
17751 1-98. Cigarette(s)
262 99. Unknown
25080 BL. NA, never or unknown if ever smoked 100+ cigarettes```